In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%pprint
%matplotlib inline
import matplotlib.pyplot as plt

# Use the following only if you are on a high definition device
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

import re
from math import cos, sin, pi, sqrt, atan
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow

%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 Image	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 atan	 attempt_to_pickle	 
cos	 example_iterrows	 get_column_descriptions	 imshow	 load_csv	 load_dataframes	 load_object	 math	 np	 
os	 pd	 pi	 pickle	 plt	 re	 save_dataframes	 sin	 sqrt	 
store_objects	 sys	 


In [2]:

cw_history_year_dict = load_object('cw_history_year_dict')
dataframes_dict = load_dataframes(patriline_df='patriline_df', turnings_df='turnings_df')

Attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\patriline_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\csv\patriline_df.csv.
Attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\pickle\turnings_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\Strauss–Howe\saves\csv\turnings_df.csv.


In [3]:

index_name = 'Patriarch Name'
patriline_df = dataframes_dict['patriline_df']
if 'Unnamed: 0' in patriline_df.columns:
    patriline_df = patriline_df.set_index('Unnamed: 0')
elif index_name in patriline_df.columns:
    patriline_df = patriline_df.set_index(index_name)
patriline_df.index.name = index_name

In [4]:

index_name = 'turning_name'
turnings_df = dataframes_dict['turnings_df']
if 'Unnamed: 0' in turnings_df.columns:
    turnings_df = turnings_df.set_index('Unnamed: 0')
elif index_name in turnings_df.columns:
    turnings_df = turnings_df.set_index(index_name)
turnings_df.index.name = index_name

In [5]:

def polar_to_cartesian(r, theta):
    radians = theta*(pi/180)
    
    return int(r*cos(radians)), int(r*sin(radians))

In [6]:

def get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i=0):
    xy_list = []
    i = i % 4
    if stop_year > stopped_year:
        stop_year = stopped_year
    start_radius = history_year_dict[start_year][0]
    start_radius += 25*i
    stop_radius = history_year_dict[stop_year][0]
    stop_radius += 25*i
    increment_count = int(2*pi*start_radius)
    radius_array = np.linspace(start=start_radius, stop=stop_radius,
                               num=increment_count)
    start_theta = history_year_dict[start_year][1]
    stop_theta = history_year_dict[stop_year][1]
    theta_array = np.linspace(start=start_theta, stop=stop_theta,
                              num=increment_count)
    for radius, theta in zip(radius_array, theta_array):
        cartesian_tuple = polar_to_cartesian(radius, theta)
        if len(xy_list):
            if (cartesian_tuple != xy_list[-1]):
                xy_list.append(cartesian_tuple)
        else:
            xy_list.append(cartesian_tuple)
    
    return xy_list

In [7]:

py_file_header_str = '''
import bpy

# Get spiral data
patriarch_coords_dict = {}
'''

In [8]:

py_file_footer_str = """

for patriarch_name, coords in patriarch_coords_dict.items():
    
    # create the Curve Datablock
    curveData = bpy.data.curves.new(patriarch_name, type='CURVE')
    curveData.dimensions = '3D'
    curveData.resolution_u = 2
    
    # map coords to spline
    polyline = curveData.splines.new('NURBS')
    polyline.points.add(len(coords))
    for i, coord in enumerate(coords):
        x,y,z = coord
        polyline.points[i].co = (x, y, z, 1)

    # create Object
    curveOB = bpy.data.objects.new(patriarch_name, curveData)

    # attach to scene and validate context
    scn = bpy.context.scene
    scn.objects.link(curveOB)
    scn.objects.active = curveOB
    curveOB.select = True"""

In [18]:

py_bezier_file_footer_str = """

for patriarch_name, coords in patriarch_coords_dict.items():
    
    # create the Curve Datablock
    curveData = bpy.data.curves.new(patriarch_name, type='CURVE')
    curveData.dimensions = '3D'
    curveData.resolution_u = 2
    
    # map coords to spline
    polyline = curveData.splines.new('BEZIER')
    coords_count = len(coords)
    polyline.bezier_points.add(coords_count-1)
    for i, coord in enumerate(coords):
        x, y, z = coord
        polyline.bezier_points[i].co = (x, y, z)
    for i in range(coords_count):
        if (i == 0) or (i == coords_count-1):
            polyline.bezier_points[i].handle_left = polyline.bezier_points[i].handle_right = polyline.bezier_points[i].co
        else:
            x, y, z = coords[i-1]
            polyline.bezier_points[i].handle_left = (x, y, z)
            x, y, z = coords[i+1]
            polyline.bezier_points[i].handle_right = (x, y, z)

    # create Object
    curveOB = bpy.data.objects.new(patriarch_name, curveData)

    # attach to scene and validate context
    scn = bpy.context.scene
    scn.objects.link(curveOB)
    scn.objects.active = curveOB
    curveOB.select = True"""

In [14]:

def save_stopped_babbitt_plot(history_year_dict, stopped_year, out_file_path, footer_str):
    mask_series = (patriline_df['Year of Birth'] <= stopped_year)
    i = patriline_df[mask_series].shape[0]-1
    with open(out_file_path, 'w') as output:
        size = output.write(py_file_header_str)
        for patriarch_name, row_series in patriline_df[mask_series].iterrows():
            start_year = int(row_series['Year of Birth'])
            stop_year = row_series['Year of Death']
            try:
                stop_year = int(stop_year)
            except:
                stop_year = start_year + 80
                if stop_year > max(history_year_dict.keys()):
                    stop_year = max(history_year_dict.keys())
            xy_list = get_one_stopped_arc(history_year_dict, start_year, stop_year, stopped_year, i)
            size = output.write("patriarch_coords_dict['{}'] = {}\n".format(patriarch_name,
                                                                            str([(x,y,1) for (x, y) in xy_list])))
            i -= 1
        size = output.write(footer_str)

In [19]:

stopped_year = 2030
out_file_path = os.path.join(py_dir, 'output_{}.py'.format(stopped_year))
save_stopped_babbitt_plot(cw_history_year_dict, stopped_year, out_file_path, py_bezier_file_footer_str)

In [13]:

py_dir = os.path.join('..', 'py')
os.makedirs(name=py_dir, exist_ok=True)
min_year = patriline_df['Year of Birth'].min()
for stopped_year in range(min_year, 2031):
    out_file_path = os.path.join(py_dir, 'output_{}.py'.format(stopped_year))
    save_stopped_babbitt_plot(cw_history_year_dict, stopped_year, out_file_path, py_file_footer_str)

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body;
var attribs = body.attributes;
var command = "notebook_path = " + "'"+attribs['data-notebook-path'].value+"'";
kernel.execute(command);

In [ ]:
# %load ../../load_magic/nbviewer.py

from IPython.display import HTML

notebook_viewer_url = 'https://nbviewer.jupyter.org/github/dbabbitt/notebooks/blob/master/'
notebook_viewer_url += '/'.join(notebook_path.split('/')[1:])
html_str = 'Click <a href="{}" target="_blank">here</a> to view notebook in nbviewer.'
HTML(html_str.format(notebook_viewer_url))